<a href="https://colab.research.google.com/github/mayourbukhari/AI-and-ML-Resources/blob/main/Mohsin_Bukhari_OCR_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
andrianang_imgur5k_sample_path = kagglehub.dataset_download('andrianang/imgur5k-sample')
hongvnphi_iam_handwriting_preprocessed_path = kagglehub.dataset_download('hongvnphi/iam-handwriting-preprocessed')

print('Data source import complete.')


100%|██████████| 809M/809M [00:40<00:00, 21.2MB/s]

Extracting files...


100%|██████████| 933M/933M [00:47<00:00, 20.6MB/s]

Extracting files...


Data source import complete.


"""
# Fine-Tuning TrOCR for Handwriting Recognition
This notebook implements the fine-tuning of Microsoft's TrOCR model for handwritten text recognition,
using the IAM Handwriting Database and Imgur5K datasets to achieve high accuracy in OCR tasks.
"""

In [3]:
# Part 1: Install necessary packages and dependencies
!pip install numpy pandas Pillow torch torchvision
!pip install transformers datasets evaluate jiwer wandb
!pip install opencv-python  # For advanced image preprocessing

In [4]:
# Part 2: Configure Weights & Biases for experiment tracking (optional)
import os
import wandb

# Set wandb API key (replace with your actual key or comment out if not using wandb)
wandb_api_key = "YOUR_WANDB_API_KEY"  # Replace with your key or load from environment
os.environ["WANDB_API_KEY"] = wandb_api_key
wandb.login(key=os.environ["WANDB_API_KEY"])


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: smayour82 (smayour82-lovely-professional-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [5]:
# Part 3: Load and preprocess datasets
import os
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import cv2  # OpenCV for advanced image preprocessing
from PIL import Image
# -------------------------------
# PART 3A: PROCESS IMGUR5K DATASET
# -------------------------------
def load_imgur_labels(labels_file, images_dir, sep="\t"):
    """
    Reads a labels file where each line contains a relative image path and its transcription.
    Returns a DataFrame with columns: image_filename, text, image_path.
    Expected format:
      ./eval_images\\image_0.png It
      ./eval_images\\image_1.png takes
      ...
    """
    data = []
    with open(labels_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    for line in lines:
        line = line.strip()
        if not line:
            continue
        tokens = line.split()
        if len(tokens) < 2:
            continue
        # Normalize path
        rel_path = tokens[0].replace("\\", "/")
        filename = os.path.basename(rel_path)
        image_path = os.path.join(images_dir, filename)
        text = " ".join(tokens[1:])
        data.append({"image_filename": filename, "text": text, "image_path": image_path})
    return pd.DataFrame(data)

# Base path for IMGUR5K dataset on Kaggle
imgur_base_path = "/kaggle/input/imgur5k-sample/"
train_images_dir = os.path.join(imgur_base_path, "train_images")
test_images_dir  = os.path.join(imgur_base_path, "test_images")
eval_images_dir  = os.path.join(imgur_base_path, "eval_images")

train_labels_file = os.path.join(imgur_base_path, "train_labels.txt")
test_labels_file  = os.path.join(imgur_base_path, "test_labels.txt")
eval_labels_file  = os.path.join(imgur_base_path, "eval_labels.txt")

# Load IMGUR5K data
df_imgur_train = load_imgur_labels(train_labels_file, train_images_dir)
df_imgur_test  = load_imgur_labels(test_labels_file, test_images_dir)
df_imgur_eval  = load_imgur_labels(eval_labels_file, eval_images_dir)

print("IMGUR5K splits:")
print("Train:", df_imgur_train.shape)
print("Test:", df_imgur_test.shape)
print("Eval:", df_imgur_eval.shape)
# -------------------------------
# PART 3B: PROCESS IAM HANDWRITING DATASET
# -------------------------------
import kagglehub
iam_path = kagglehub.dataset_download("hongvnphi/iam-handwriting-preprocessed")
# Adjust the IAM path if needed:
iam_path = iam_path + '/kaggle/working/iam-handwriting-dataset/'
print("Path to IAM dataset files:", iam_path)

def load_iam_preprocessed(split_folder):
    """
    Reads image and label files from a split folder.
    Expects 'images' and 'labels' subfolders.
    Returns a DataFrame with columns: image_filename, text, image_path.
    """
    images_dir = os.path.join(split_folder, "images")
    labels_dir = os.path.join(split_folder, "labels")
    label_files = [f for f in os.listdir(labels_dir) if f.endswith(".txt")]
    data = []
    for label_file in label_files:
        base = os.path.splitext(label_file)[0]
        image_filename = base + ".png"
        image_path = os.path.join(images_dir, image_filename)
        label_path = os.path.join(labels_dir, label_file)
        with open(label_path, "r", encoding="utf-8") as f:
            label_text = f.read().strip()
        data.append({
            "image_filename": image_filename,
            "text": label_text,
            "image_path": image_path
        })
    return pd.DataFrame(data)

iam_dataset_path = iam_path
df_iam_train = load_iam_preprocessed(os.path.join(iam_dataset_path, "train"))
df_iam_val   = load_iam_preprocessed(os.path.join(iam_dataset_path, "validation"))
df_iam_test  = load_iam_preprocessed(os.path.join(iam_dataset_path, "test"))

print("IAM splits:")
print("Train:", df_iam_train.shape)
print("Validation:", df_iam_val.shape)
print("Test:", df_iam_test.shape)

# -------------------------------
# PART 3C: COMBINE DATASETS AND SPLIT FOR FINE-TUNING
# -------------------------------
# Combine all splits from both datasets
df_imgur_all = pd.concat([df_imgur_train, df_imgur_test, df_imgur_eval], ignore_index=True)
df_iam_all   = pd.concat([df_iam_train, df_iam_val, df_iam_test], ignore_index=True)

# Following the assignment requirements for test set evaluation
# We'll keep the IAM test set separate for final evaluation
df_iam_test_reserved = df_iam_test.copy()
df_iam_rest = pd.concat([df_iam_train, df_iam_val], ignore_index=True)

# Combine all training data (IAM rest + IMGUR5K)
df_train_combined = pd.concat([df_iam_rest, df_imgur_all], ignore_index=True)

# Shuffle the combined data
df_train_combined = df_train_combined.sample(frac=1, random_state=42).reset_index(drop=True)

# Split into train (90%) and validation (10%) following assignment requirements
train_size = 0.9
df_train, df_val = train_test_split(df_train_combined, test_size=(1-train_size), random_state=42)

print("Final split shapes:")
print("Train:", df_train.shape)
print("Validation:", df_val.shape)
print("Test (IAM test set):", df_iam_test_reserved.shape)

# Save the splits as CSV files
df_train.to_csv("combined_train.csv", index=False)
df_val.to_csv("combined_val.csv", index=False)
df_iam_test_reserved.to_csv("iam_test.csv", index=False)
print("Saved CSV files: combined_train.csv, combined_val.csv, iam_test.csv")


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/imgur5k-sample/train_labels.txt'

In [ ]:
# Part 4: Define the OCR dataset and model architecture
import torch
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from transformers import (
    VisionEncoderDecoderModel,
    TrOCRProcessor,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    TrainerCallback,
    EarlyStoppingCallback
)
import evaluate
from torch.cuda.amp import autocast  # For mixed precision training

# Set seed for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)

# Configuration parameters as per assignment requirements
class Config:
    BATCH_SIZE = 4                # For Tesla P100 or single T4
    EPOCHS = 10                   # Minimum 10 epochs as per assignment
    LEARNING_RATE = 5e-5          # Suggested learning rate in assignment
    TRAIN_CSV = "combined_train.csv"
    VAL_CSV = "combined_val.csv"
    TEST_CSV = "iam_test.csv"     # Using IAM test set for final evaluation
    IMAGE_SIZE = (384, 384)       # 384x384 as suggested in the assignment
    USE_MIXED_PRECISION = True    # Enable mixed precision training
    GRADIENT_ACCUMULATION_STEPS = 2  # To simulate larger batch sizes
    EARLY_STOPPING_PATIENCE = 3   # Stop training if no improvement after 3 epochs
    CER_TARGET = 0.07             # 7% target CER
    WER_TARGET = 0.15             # 15% target WER

# Enhanced OCR Dataset with more robust image preprocessing
class OCRDataset(Dataset):
    def __init__(self, csv_file, processor, transform=None, max_samples=None, advanced_preprocessing=True):
        self.data = pd.read_csv(csv_file)
        if max_samples is not None:
            self.data = self.data.head(max_samples)
        # Ensure only valid rows with non-null image paths are kept
        self.data = self.data[self.data["image_path"].notnull() & (self.data["image_path"] != "")]
        self.processor = processor
        self.transform = transform
        self.advanced_preprocessing = advanced_preprocessing

    def __len__(self):
        return len(self.data)

    def preprocess_image(self, image):
        """Advanced image preprocessing with OpenCV"""
        if not self.advanced_preprocessing:
            return image

        # Convert PIL to OpenCV format
        img = np.array(image)

        # Apply noise reduction (Gaussian blur)
        img = cv2.GaussianBlur(img, (3, 3), 0)

        # Apply adaptive thresholding for better contrast in handwritten text
        if len(img.shape) == 3:  # Color image
            img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        else:  # Already grayscale
            img_gray = img

        # Convert back to RGB (3 channels) as TrOCR expects RGB input
        img_processed = cv2.cvtColor(img_gray, cv2.COLOR_GRAY2RGB)

        # Convert back to PIL
        return Image.fromarray(img_processed)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        image_path = row["image_path"]

        try:
            # Open image and apply preprocessing
            image = Image.open(image_path).convert("RGB")
            image = self.preprocess_image(image)

            if self.transform:
                image = self.transform(image)

            text = str(row["text"])
            return {"image": image, "text": text}
        except Exception as e:
            print(f"Error processing image {image_path}: {e}")
            # Return a placeholder to avoid breaking the batch
            # In a production system, you might want to handle this differently
            placeholder = Image.new('RGB', Config.IMAGE_SIZE, color='white')
            if self.transform:
                placeholder = self.transform(placeholder)
            return {"image": placeholder, "text": ""}

# Enhanced data augmentation for training
train_transforms = transforms.Compose([
    transforms.Resize(Config.IMAGE_SIZE),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.RandomRotation(degrees=5),  # More conservative rotation to avoid distorting text
    transforms.RandomAffine(degrees=0, translate=(0.05, 0.05), scale=(0.95, 1.05)),  # Slight affine transforms
    transforms.ToTensor(),
])

val_transforms = transforms.Compose([
    transforms.Resize(Config.IMAGE_SIZE),
    transforms.ToTensor(),
])

# Load pre-trained TrOCR model and processor
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-large-handwritten")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-large-handwritten")

# Set device and move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Enable multi-GPU if available
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs with DataParallel.")
    model = torch.nn.DataParallel(model)

# Set special tokens if not already defined
if model.config.decoder_start_token_id is None:
    model.config.decoder_start_token_id = processor.tokenizer.bos_token_id
if model.config.pad_token_id is None:
    model.config.pad_token_id = processor.tokenizer.pad_token_id

# Create datasets
train_dataset = OCRDataset(Config.TRAIN_CSV, processor, transform=train_transforms)
val_dataset = OCRDataset(Config.VAL_CSV, processor, transform=val_transforms)
test_dataset = OCRDataset(Config.TEST_CSV, processor, transform=val_transforms)

# Enhanced data collator with better error handling
def data_collator(features):
    """Collate features into batch"""
    # Filter out any samples missing required keys or with empty values
    valid_features = [f for f in features if "image" in f and "text" in f and f["image"] is not None and f["text"] is not None]
    if len(valid_features) < len(features):
        print(f"Warning: Filtered out {len(features) - len(valid_features)} invalid samples from batch.")
    if not valid_features:
        # In a real-world scenario, you might need more robust handling of completely invalid batches
        raise ValueError("All samples in the batch are invalid.")

    images = [f["image"] for f in valid_features]
    texts  = [f["text"] for f in valid_features]

    # Process images and texts using the processor in batch
    try:
        pixel_values = processor(images=images, return_tensors="pt", do_rescale=False).pixel_values
    except Exception as e:
        print(f"Error processing images: {e}")
        # Create a placeholder batch with correct dimensions in case of processing errors
        pixel_values = torch.zeros((len(images), 3, Config.IMAGE_SIZE[0], Config.IMAGE_SIZE[1]), dtype=torch.float32)

    # Process text with padding and truncation
    labels = processor.tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt"
    ).input_ids

    # Replace padding token ids with -100 so they're ignored by the loss function
    labels[labels == processor.tokenizer.pad_token_id] = -100

    return {"pixel_values": pixel_values, "labels": labels}

# Evaluation metrics
cer_metric = evaluate.load("cer")
wer_metric = evaluate.load("wer")

def compute_metrics(eval_pred):
    """Compute CER and WER metrics"""
    predictions, labels = eval_pred
    # Replace -100 with pad token id for decoding
    predictions = np.where(predictions == -100, processor.tokenizer.pad_token_id, predictions)
    labels = np.where(labels == -100, processor.tokenizer.pad_token_id, labels)

    # Decode predictions and references
    pred_str = processor.batch_decode(predictions, skip_special_tokens=True)
    label_str = processor.batch_decode(labels, skip_special_tokens=True)

    # Remove empty predictions/references to avoid metric calculation errors
    filtered_pairs = [(p, l) for p, l in zip(pred_str, label_str) if p.strip() and l.strip()]
    if not filtered_pairs:
        return {"cer": 1.0, "wer": 1.0}  # Return max error if no valid pairs

    filtered_preds, filtered_labels = zip(*filtered_pairs)

    # Compute metrics
    cer = cer_metric.compute(predictions=filtered_preds, references=filtered_labels)
    wer = wer_metric.compute(predictions=filtered_preds, references=filtered_labels)

    return {"cer": cer, "wer": wer}

# Custom logging callback for detailed training progress
class LoggingCallback(TrainerCallback):
    """Callback for detailed training logging"""
    def on_train_begin(self, args, state, control, **kwargs):
        print(f"==== Starting training on {device} ====")
        if torch.cuda.is_available():
            print(f"GPU: {torch.cuda.get_device_name(0)}")
            print(f"Memory allocated: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB")
        print(f"Training {Config.EPOCHS} epochs with batch size {Config.BATCH_SIZE}")
        if Config.USE_MIXED_PRECISION:
            print("Using mixed precision training")

    def on_epoch_begin(self, args, state, control, **kwargs):
        print(f"Epoch {state.epoch:.2f} starting...")

    def on_epoch_end(self, args, state, control, **kwargs):
        # Log metrics from the most recent log history entry
        if state.log_history:
            last_metrics = {}
            # Extract metrics from the end of log history
            for entry in reversed(state.log_history):
                if 'eval_cer' in entry:
                    last_metrics = entry
                    break

            if last_metrics:
                print(f"Epoch {state.epoch:.2f} finished. Metrics:")
                print(f"  CER: {last_metrics.get('eval_cer', 'N/A'):.4f}")
                print(f"  WER: {last_metrics.get('eval_wer', 'N/A'):.4f}")
                print(f"  Loss: {last_metrics.get('eval_loss', 'N/A'):.4f}")

                # Check if we've reached target metrics
                if last_metrics.get('eval_cer', 1.0) <= Config.CER_TARGET and \
                   last_metrics.get('eval_wer', 1.0) <= Config.WER_TARGET:
                    print(f"✅ Target metrics achieved! CER: {last_metrics.get('eval_cer'):.4f}, WER: {last_metrics.get('eval_wer'):.4f}")
            else:
                print(f"Epoch {state.epoch:.2f} finished, but no metrics available.")

    def on_train_end(self, args, state, control, **kwargs):
        print("==== Training completed ====")


2025-05-16 16:32:20.763014: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747413140.785034     175 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747413140.791854     175 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.23G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.23G [00:00<?, ?B/s]

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 16,
  "num_channels": 3,
  "num_hidden_layers": 24,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 1024,
  "qkv_bias": false,
  "torch_dtype": "float32",
  "transformers_version": "4.51.3"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 1024,
  "d_

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Part 5: Configure training arguments and trainer
training_args = Seq2SeqTrainingArguments(
    output_dir="./trocr_finetuned",
    per_device_train_batch_size=Config.BATCH_SIZE,
    per_device_eval_batch_size=Config.BATCH_SIZE,
    num_train_epochs=Config.EPOCHS,
    eval_strategy="epoch",  # Changed from evaluation_strategy to eval_strategy
    save_strategy="epoch",
    learning_rate=Config.LEARNING_RATE,
    fp16=Config.USE_MIXED_PRECISION,  # Mixed precision for memory efficiency
    gradient_accumulation_steps=Config.GRADIENT_ACCUMULATION_STEPS,  # Simulate larger batch sizes
    logging_steps=100,
    logging_first_step=True,
    log_level="info",
    save_total_limit=2,  # Keep only the 2 best checkpoints
    predict_with_generate=True,
    dataloader_num_workers=4,  # Parallel data loading
    report_to=["wandb"] if "WANDB_API_KEY" in os.environ else ["none"],  # Only use wandb if API key is set
    logging_dir="./logs",
    run_name="trocr_finetuning_handwriting",
    remove_unused_columns=False,
    load_best_model_at_end=True,
    metric_for_best_model="wer",  # Monitor WER for best model selection
    greater_is_better=False,  # Lower WER is better
)

# Initialize trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Add callbacks for logging and early stopping
trainer.add_callback(LoggingCallback())
trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=Config.EARLY_STOPPING_PATIENCE))

PyTorch: setting up devices
Using auto half precision backend


In [ ]:
# Part 6: Train the model
print("Starting fine-tuning...")
trainer.train()

# Save the fine-tuned model and processor
trainer.save_model("./trocr_finetuned_final")
processor.save_pretrained("./trocr_finetuned_final")
print("Model saved to ./trocr_finetuned_final")

***** Running training *****
  Num examples = 20,212
  Num Epochs = 10
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 2
  Total optimization steps = 25,260
  Number of trainable parameters = 558,226,432
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Starting fine-tuning...


==== Starting training on cuda ====
GPU: Tesla P100-PCIE-16GB
Memory allocated: 2.23 GB
Training 10 epochs with batch size 4
Using mixed precision training
Epoch 0.00 starting...


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss


In [ ]:
# Part 7: Evaluate on test set (IAM test set)
print("\n===== Evaluating on IAM test set =====")
test_metrics = trainer.evaluate(test_dataset)
print("Test Metrics:", test_metrics)

# Check if target metrics are achieved
target_achieved = (test_metrics["eval_cer"] <= Config.CER_TARGET and
                  test_metrics["eval_wer"] <= Config.WER_TARGET)

if target_achieved:
    print(f"🎉 Target metrics achieved: CER <= {Config.CER_TARGET} and WER <= {Config.WER_TARGET}")
else:
    print(f"Target metrics not achieved. CER: {test_metrics['eval_cer']:.4f}, WER: {test_metrics['eval_wer']:.4f}")



In [ ]:
# Part 8: Define and demonstrate inference function
def ocr_inference(image_path, preprocess=True):
    """
    Perform OCR inference on a single image

    Args:
        image_path: Path to the image file
        preprocess: Whether to apply advanced preprocessing

    Returns:
        Generated text from the OCR model
    """
    # Load and preprocess image
    image = Image.open(image_path).convert("RGB")

    # Apply advanced preprocessing if requested
    if preprocess:
        # Convert to OpenCV format for preprocessing
        img = np.array(image)
        # Apply noise reduction
        img = cv2.GaussianBlur(img, (3, 3), 0)
        # Convert to grayscale for better text recognition
        img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) if len(img.shape) == 3 else img
        # Convert back to RGB (3 channels) for the model
        img = cv2.cvtColor(img_gray, cv2.COLOR_GRAY2RGB)
        # Convert back to PIL
        image = Image.fromarray(img)

    # Resize image
    image = image.resize(Config.IMAGE_SIZE)
    # Convert to tensor
    image = transforms.ToTensor()(image)

    # Process with TrOCR processor and generate text
    pixel_values = processor(images=image, return_tensors="pt", do_rescale=False).pixel_values.to(device)

    # Generate text with the model
    with torch.no_grad():
        generated_ids = model.generate(pixel_values)

    # Decode the generated text
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return generated_text

# Sample inference demonstration
from IPython.display import display

# Perform inference on sample images from the test set
sample_count = 5
print(f"\n===== Sample OCR Inference on {sample_count} Test Images =====")

for idx, row in pd.read_csv(Config.TEST_CSV).head(sample_count).iterrows():
    image_path = row["image_path"]
    ground_truth = row["text"]

    try:
        # Open and display the test image
        image = Image.open(image_path).convert("RGB")
        print(f"\nTest Image {idx+1}:")
        display(image)

        # Run OCR inference
        predicted_text = ocr_inference(image_path)

        # Print ground truth and prediction
        print(f"Ground Truth: {ground_truth}")
        print(f"OCR Output:   {predicted_text}")

        # Calculate CER and WER for this prediction
        sample_cer = cer_metric.compute(predictions=[predicted_text], references=[ground_truth])
        sample_wer = wer_metric.compute(predictions=[predicted_text], references=[ground_truth])
        print(f"Sample CER: {sample_cer:.4f}, Sample WER: {sample_wer:.4f}")

    except Exception as e:
        print(f"Error processing image {image_path}: {e}")

print("\n===== Fine-tuning Complete =====")
print(f"Final CER: {test_metrics['eval_cer']:.4f} (Target: {Config.CER_TARGET})")
print(f"Final WER: {test_metrics['eval_wer']:.4f} (Target: {Config.WER_TARGET})")
print(f"{'✅ Target achieved' if target_achieved else '❌ Target not achieved'}")